In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install optuna
!pip install shap
!pip install pytorch-tabnet
!pip install tabpfn

In [ ]:
import numpy as np
import pandas as pd

import sklearn
import sklearn.metrics
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score, auc, precision_score, recall_score, accuracy_score, balanced_accuracy_score, brier_score_loss, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from math import sqrt
from scipy import stats as st
from random import randrange

from matplotlib import pyplot
import seaborn as sns

from tabpfn import TabPFNClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import optuna
from optuna.samplers import TPESampler

import shap
from sklearn.inspection import PartialDependenceDisplay

# Preparing Data

In [ ]:
#Open csv file.

data = pd.read_csv('/content/drive/MyDrive/NCDB-Meningioma/final_data.csv', index_col = 0)

In [ ]:
#Define outcome of interest (3-month mortality).

data.loc[(data['Vital Status'] == 'Dead') & (data['Last Contact or Death (Months from Diagnosis)'] < 60), 'OUTCOME'] = 1
data.loc[(data['Vital Status'] == 'Dead') & (data['Last Contact or Death (Months from Diagnosis)'] >= 60), 'OUTCOME'] = 0
data.loc[(data['Vital Status'] == 'Alive') & (data['Last Contact or Death (Months from Diagnosis)'] >= 60), 'OUTCOME'] = 0
data = data.dropna(subset=['OUTCOME'])

print(data['OUTCOME'].value_counts(normalize=False, dropna=False))
print(data['OUTCOME'].value_counts(normalize=True, dropna=False))

In [ ]:
#Define predictor variables (x) and outcome of interest (y).

outcomes = ['Vital Status', 'Last Contact or Death (Months from Diagnosis)', 'OUTCOME']

x = data.drop(outcomes, axis = 1)
y = data['OUTCOME']

In [ ]:
#Check data shapes.

print(y.shape)
print(x.shape)

In [ ]:
#Split data into initial train set and test set in 80:20 ratio.

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

#Describe initial train set and test set.

print("Number patients x_train dataset: ", x_train.shape[0])
print("Number patients y_train dataset: ", y_train.shape[0])
print("Number patients x_test dataset: ", x_test.shape[0])
print("Number patients y_test dataset: ", y_test.shape[0])

In [ ]:
#Split initial train set into final train set and validation set in 75:25 ratio.

train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_train, test_size = 0.25, random_state = 0)

#Describe train and validation sets.

print("Number patients train_x dataset: ", train_x.shape[0])
print("Number patients train_y dataset: ", train_y.shape[0])
print("Number patients valid_x dataset: ", valid_x.shape[0])
print("Number patients valid_y dataset: ", valid_y.shape[0])

In [ ]:
#Describe outcome of interest before resampling.

print("Before resampling, counts of label '1': {}".format(sum(train_y == 1)))
print("Before resampling, counts of label '0': {} \n".format(sum(train_y == 0)))

In [ ]:
#Apply SMOTE.

from imblearn.over_sampling import SMOTE

resampler = SMOTE(random_state = 0)
train_x, train_y = resampler.fit_resample(train_x, train_y)

In [ ]:
#Describe outcome of interest after resampling.

print("After resampling, counts of label '1': {}".format(sum(train_y == 1)))
print("After resampling, counts of label '0': {} \n".format(sum(train_y == 0)))

In [ ]:
#Define function for AUROC with 95% confidence intervals.

def auroc_ci(y_test, y_probs, positive=1, n_samples=1000, alpha=0.05):
    auroc_values = []
    for i in range(n_samples):
        y_test_bs, y_probs_bs = resample(y_test, y_probs)

        auroc = roc_auc_score(y_test_bs, y_probs_bs)
        auroc_values.append(auroc)

    mean_auroc = np.mean(auroc_values)
    std_auroc = np.std(auroc_values)

    lower_ci = mean_auroc - 1.96 * std_auroc
    upper_ci = mean_auroc + 1.96 * std_auroc

    return auroc, lower_ci, upper_ci

In [ ]:
#Define z-value for other confidence intervals.

confidence = 0.95
z_value = st.norm.ppf((1 + confidence) / 2.0)

#TabPFN

No hyperparameter tuning is performed for TabPFN since the paper that introduced TabPFN claims no hyperparameter tuning is needed for it:


*We present TabPFN, a trained Transformer that can do supervised classification for small tabular datasets in less than a second, needs no hyperparameter tuning and is competitive with state-of-the-art classification methods.*

https://doi.org/10.48550/arXiv.2207.01848


In [ ]:
#Fit TabPFN.

tabpfn = TabPFNClassifier(device='cpu', N_ensemble_configurations=2)

tabpfn.fit(x_train.sample(n=256, random_state=42), y_train.sample(n=256, random_state=42), overwrite_warning = True)

In [ ]:
#Predict on the validation set, get predicted probabilities for calibration, and fit calibration function.

calib_probs_tabpfn = tabpfn.predict_proba(valid_x.values)

calib_model_tabpfn = LogisticRegression()
calib_model_tabpfn.fit(calib_probs_tabpfn, valid_y)

In [ ]:
#Make predictions on the test set based on the trained TabNet model.

preds_tabpfn = tabpfn.predict(x_test.values)

uncalibrated_probs_tabpfn = tabpfn.predict_proba(x_test.values)

probs_tabpfn = calib_model_tabpfn.predict_proba(uncalibrated_probs_tabpfn)
probs_tabpfn = probs_tabpfn[:, 1]

In [ ]:
#Evaluate TabPFN model.

tabpfn_tn, tabpfn_fp, tabpfn_fn, tabpfn_tp = confusion_matrix(y_test, preds_tabpfn).ravel()
tabpfn_cm = confusion_matrix(y_test, preds_tabpfn)

tabpfn_sensitivity = round(tabpfn_tp/(tabpfn_tp+tabpfn_fn), 3)
tabpfn_sensitivity_ci_length = z_value * np.sqrt((tabpfn_sensitivity * (1 - tabpfn_sensitivity)) / y_test.shape[0])
tabpfn_sensitivity_ci_lower = round((tabpfn_sensitivity - tabpfn_sensitivity_ci_length), 3)
tabpfn_sensitivity_ci_upper = round((tabpfn_sensitivity + tabpfn_sensitivity_ci_length), 3)
tabpfn_sensitivity_str = str(tabpfn_sensitivity) + ' (' + str(tabpfn_sensitivity_ci_lower) + ' - ' + str(tabpfn_sensitivity_ci_upper) + ')'

tabpfn_specificity = round(tabpfn_tn/(tabpfn_tn+tabpfn_fp), 3)
tabpfn_specificity_ci_length = z_value * np.sqrt((tabpfn_specificity * (1 - tabpfn_specificity)) / y_test.shape[0])
tabpfn_specificity_ci_lower = round((tabpfn_specificity - tabpfn_specificity_ci_length), 3)
tabpfn_specificity_ci_upper = round((tabpfn_specificity + tabpfn_specificity_ci_length), 3)
tabpfn_specificity_str = str(tabpfn_specificity) + ' (' + str(tabpfn_specificity_ci_lower) + ' - ' + str(tabpfn_specificity_ci_upper) + ')'

tabpfn_auprc = round(average_precision_score(y_test, probs_tabpfn), 3)
tabpfn_auprc_ci_length = z_value * np.sqrt((tabpfn_auprc * (1 - tabpfn_auprc)) / y_test.shape[0])
tabpfn_auprc_ci_lower = round((tabpfn_auprc - tabpfn_auprc_ci_length), 3)
tabpfn_auprc_ci_upper = round((tabpfn_auprc + tabpfn_auprc_ci_length), 3)
tabpfn_auprc_str = str(tabpfn_auprc) + ' (' + str(tabpfn_auprc_ci_lower) + ' - ' + str(tabpfn_auprc_ci_upper) + ')'

tabpfn_accuracy = round(accuracy_score(y_test, preds_tabpfn), 3)
tabpfn_accuracy_ci_length = z_value * np.sqrt((tabpfn_accuracy * (1 - tabpfn_accuracy)) / y_test.shape[0])
tabpfn_accuracy_ci_lower = round((tabpfn_accuracy - tabpfn_accuracy_ci_length), 3)
tabpfn_accuracy_ci_upper = round((tabpfn_accuracy + tabpfn_accuracy_ci_length), 3)
tabpfn_accuracy_str = str(tabpfn_accuracy) + ' (' + str(tabpfn_accuracy_ci_lower) + ' - ' + str(tabpfn_accuracy_ci_upper) + ')'

tabpfn_auroc, tabpfn_auroc_ci_lower, tabpfn_auroc_ci_upper = auroc_ci(y_test, probs_tabpfn)
tabpfn_auroc = round(tabpfn_auroc, 3)
tabpfn_auroc_ci_lower = round(tabpfn_auroc_ci_lower, 3)
tabpfn_auroc_ci_upper = round(tabpfn_auroc_ci_upper, 3)
tabpfn_auroc_str = str(tabpfn_auroc) + ' (' + str(tabpfn_auroc_ci_lower) + ' - ' + str(tabpfn_auroc_ci_upper) + ')'

tabpfn_brier = round(brier_score_loss(y_test, probs_tabpfn), 3)
tabpfn_brier_ci_length = z_value * np.sqrt((tabpfn_brier * (1 - tabpfn_brier)) / y_test.shape[0])
tabpfn_brier_ci_lower = round((tabpfn_brier - tabpfn_brier_ci_length), 3)
tabpfn_brier_ci_upper = round((tabpfn_brier + tabpfn_brier_ci_length), 3)
tabpfn_brier_str = str(tabpfn_brier) + ' (' + str(tabpfn_brier_ci_lower) + ' - ' + str(tabpfn_brier_ci_upper) + ')'

tabpfn_results = [tabpfn_sensitivity_str, tabpfn_specificity_str, tabpfn_auprc_str, tabpfn_accuracy_str, tabpfn_auroc_str, tabpfn_brier_str]

print("Sensitivity: ", (tabpfn_sensitivity_str))
print("Specificity: ", (tabpfn_specificity_str))
print('AUPRC: ', (tabpfn_auprc_str))
print('Accuracy: ', (tabpfn_accuracy_str))
print('AUROC: ', (tabpfn_auroc_str))
print('Brier Score: ', (tabpfn_brier_str))

In [ ]:
#Recalculate precision recall curve for plotting purposes.

tabpfn_precision_curve, tabpfn_recall_curve, _ = precision_recall_curve(y_test, probs_tabpfn)

# TabNet

In [ ]:
# Hyperparameter tuning for TabNet.

def objective(trial):

    param = {
        "n_d": trial.suggest_int("n_d", 8, 64),
        "n_a": trial.suggest_int("n_a", 8, 64),
        "n_steps": trial.suggest_int("n_steps", 1, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "n_independent": trial.suggest_int("n_independent", 1, 5),
        "n_shared": trial.suggest_int("n_shared", 1, 5),
        "lambda_sparse": trial.suggest_loguniform("lambda_sparse", 1e-6, 1e-3),
        "optimizer_params": {"lr": trial.suggest_loguniform("lr", 1e-5, 1e-2)},
        "seed": 31,
    }

    tabnet_clf = TabNetClassifier(**param)

    tabnet_clf.fit(
        train_x.values,
        train_y.values,
        eval_set=[(valid_x.values, valid_y.values)],
        eval_metric=["auc"],
        max_epochs=20,
        patience=20,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
    )

    preds = tabnet_clf.predict_proba(valid_x.values)[:, 1]
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=31))
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    tabnet_params = {}

    for key, value in trial.params.items():
        tabnet_params[key] = value

In [ ]:
#See TabNet hyperparameters.

tabnet_params.pop('lr')

print(tabnet_params)

In [ ]:
#Fit TabNet.

tabnet = TabNetClassifier(**tabnet_params)

tabnet.fit(train_x.values, train_y.values)

In [ ]:
#Predict on the validation set, get predicted probabilities for calibration, and fit calibration function.

calib_probs_tabnet = tabnet.predict_proba(valid_x.values)

calib_model_tabnet = LogisticRegression()
calib_model_tabnet.fit(calib_probs_tabnet, valid_y)

In [ ]:
#Make predictions on the test set based on the trained TabNet model.

preds_tabnet = tabnet.predict(x_test.values)

uncalibrated_probs_tabnet = tabnet.predict_proba(x_test.values)

probs_tabnet = calib_model_tabnet.predict_proba(uncalibrated_probs_tabnet)
probs_tabnet = probs_tabnet[:, 1]

In [ ]:
#Evaluate TabNet model.

tabnet_tn, tabnet_fp, tabnet_fn, tabnet_tp = confusion_matrix(y_test, preds_tabnet).ravel()
tabnet_cm = confusion_matrix(y_test, preds_tabnet)

tabnet_sensitivity = round(tabnet_tp/(tabnet_tp+tabnet_fn), 3)
tabnet_sensitivity_ci_length = z_value * np.sqrt((tabnet_sensitivity * (1 - tabnet_sensitivity)) / y_test.shape[0])
tabnet_sensitivity_ci_lower = round((tabnet_sensitivity - tabnet_sensitivity_ci_length), 3)
tabnet_sensitivity_ci_upper = round((tabnet_sensitivity + tabnet_sensitivity_ci_length), 3)
tabnet_sensitivity_str = str(tabnet_sensitivity) + ' (' + str(tabnet_sensitivity_ci_lower) + ' - ' + str(tabnet_sensitivity_ci_upper) + ')'

tabnet_specificity = round(tabnet_tn/(tabnet_tn+tabnet_fp), 3)
tabnet_specificity_ci_length = z_value * np.sqrt((tabnet_specificity * (1 - tabnet_specificity)) / y_test.shape[0])
tabnet_specificity_ci_lower = round((tabnet_specificity - tabnet_specificity_ci_length), 3)
tabnet_specificity_ci_upper = round((tabnet_specificity + tabnet_specificity_ci_length), 3)
tabnet_specificity_str = str(tabnet_specificity) + ' (' + str(tabnet_specificity_ci_lower) + ' - ' + str(tabnet_specificity_ci_upper) + ')'

tabnet_auprc = round(average_precision_score(y_test, probs_tabnet), 3)
tabnet_auprc_ci_length = z_value * np.sqrt((tabnet_auprc * (1 - tabnet_auprc)) / y_test.shape[0])
tabnet_auprc_ci_lower = round((tabnet_auprc - tabnet_auprc_ci_length), 3)
tabnet_auprc_ci_upper = round((tabnet_auprc + tabnet_auprc_ci_length), 3)
tabnet_auprc_str = str(tabnet_auprc) + ' (' + str(tabnet_auprc_ci_lower) + ' - ' + str(tabnet_auprc_ci_upper) + ')'

tabnet_accuracy = round(accuracy_score(y_test, preds_tabnet), 3)
tabnet_accuracy_ci_length = z_value * np.sqrt((tabnet_accuracy * (1 - tabnet_accuracy)) / y_test.shape[0])
tabnet_accuracy_ci_lower = round((tabnet_accuracy - tabnet_accuracy_ci_length), 3)
tabnet_accuracy_ci_upper = round((tabnet_accuracy + tabnet_accuracy_ci_length), 3)
tabnet_accuracy_str = str(tabnet_accuracy) + ' (' + str(tabnet_accuracy_ci_lower) + ' - ' + str(tabnet_accuracy_ci_upper) + ')'

tabnet_auroc, tabnet_auroc_ci_lower, tabnet_auroc_ci_upper = auroc_ci(y_test, probs_tabnet)
tabnet_auroc = round(tabnet_auroc, 3)
tabnet_auroc_ci_lower = round(tabnet_auroc_ci_lower, 3)
tabnet_auroc_ci_upper = round(tabnet_auroc_ci_upper, 3)
tabnet_auroc_str = str(tabnet_auroc) + ' (' + str(tabnet_auroc_ci_lower) + ' - ' + str(tabnet_auroc_ci_upper) + ')'

tabnet_brier = round(brier_score_loss(y_test, probs_tabnet), 3)
tabnet_brier_ci_length = z_value * np.sqrt((tabnet_brier * (1 - tabnet_brier)) / y_test.shape[0])
tabnet_brier_ci_lower = round((tabnet_brier - tabnet_brier_ci_length), 3)
tabnet_brier_ci_upper = round((tabnet_brier + tabnet_brier_ci_length), 3)
tabnet_brier_str = str(tabnet_brier) + ' (' + str(tabnet_brier_ci_lower) + ' - ' + str(tabnet_brier_ci_upper) + ')'

tabnet_results = [tabnet_sensitivity_str, tabnet_specificity_str, tabnet_auprc_str, tabnet_accuracy_str, tabnet_auroc_str, tabnet_brier_str]

print("Sensitivity: ", (tabnet_sensitivity_str))
print("Specificity: ", (tabnet_specificity_str))
print('AUPRC: ', (tabnet_auprc_str))
print('Accuracy: ', (tabnet_accuracy_str))
print('AUROC: ', (tabnet_auroc_str))
print('Brier Score: ', (tabnet_brier_str))

In [ ]:
#Recalculate precision recall curve for plotting purposes.

tabnet_precision_curve, tabnet_recall_curve, _ = precision_recall_curve(y_test, probs_tabnet)

# XGBoost

In [ ]:
#Hyperparameter tuning for XGBoost.

def objective(trial):

    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "seed": 31,
        "verbosity": 0,
        "objective":  trial.suggest_categorical("objective", ["binary:logistic"]),
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "max_depth" : trial.suggest_int("max_depth", 1, 9),
        "eta" : trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "gamma" : trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "grow_policy" : trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }


    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")

    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize", sampler = TPESampler(seed=31)
    )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    xgb_params = {}

    for key, value in trial.params.items():
        xgb_params[key] = value

In [ ]:
#See XGBoost hyperparameters.

xgb_params['eval_metric'] = 'auc'
xgb_params['verbosity'] = 0
xgb_params['seed'] = 31

print(xgb_params)

In [ ]:
#Fit XGBoost.

from xgboost import XGBClassifier

xgb = XGBClassifier(**xgb_params)

xgb.fit(train_x, train_y)

In [ ]:
#Predict on the validation set, get predicted probabilities for calibration, and fit calibration function.

calib_probs_xgb = xgb.predict_proba(valid_x)

calib_model_xgb = LogisticRegression()
calib_model_xgb.fit(calib_probs_xgb, valid_y)

In [ ]:
#Make predictions on the test set based on the trained and calibrated XGBoost model.

preds_xgb = xgb.predict(x_test)

uncalibrated_probs_xgb = xgb.predict_proba(x_test)

probs_xgb = calib_model_xgb.predict_proba(uncalibrated_probs_xgb)
probs_xgb = probs_xgb[:, 1]

In [ ]:
#Evaluate XGBoost model.

xgb_tn, xgb_fp, xgb_fn, xgb_tp = confusion_matrix(y_test, preds_xgb).ravel()
xgb_cm = confusion_matrix(y_test, preds_xgb)

xgb_sensitivity = round(xgb_tp/(xgb_tp+xgb_fn), 3)
xgb_sensitivity_ci_length = z_value * np.sqrt((xgb_sensitivity * (1 - xgb_sensitivity)) / y_test.shape[0])
xgb_sensitivity_ci_lower = round((xgb_sensitivity - xgb_sensitivity_ci_length), 3)
xgb_sensitivity_ci_upper = round((xgb_sensitivity + xgb_sensitivity_ci_length), 3)
xgb_sensitivity_str = str(xgb_sensitivity) + ' (' + str(xgb_sensitivity_ci_lower) + ' - ' + str(xgb_sensitivity_ci_upper) + ')'

xgb_specificity = round(xgb_tn/(xgb_tn+xgb_fp), 3)
xgb_specificity_ci_length = z_value * np.sqrt((xgb_specificity * (1 - xgb_specificity)) / y_test.shape[0])
xgb_specificity_ci_lower = round((xgb_specificity - xgb_specificity_ci_length), 3)
xgb_specificity_ci_upper = round((xgb_specificity + xgb_specificity_ci_length), 3)
xgb_specificity_str = str(xgb_specificity) + ' (' + str(xgb_specificity_ci_lower) + ' - ' + str(xgb_specificity_ci_upper) + ')'

xgb_auprc = round(average_precision_score(y_test, probs_xgb), 3)
xgb_auprc_ci_length = z_value * np.sqrt((xgb_auprc * (1 - xgb_auprc)) / y_test.shape[0])
xgb_auprc_ci_lower = round((xgb_auprc - xgb_auprc_ci_length), 3)
xgb_auprc_ci_upper = round((xgb_auprc + xgb_auprc_ci_length), 3)
xgb_auprc_str = str(xgb_auprc) + ' (' + str(xgb_auprc_ci_lower) + ' - ' + str(xgb_auprc_ci_upper) + ')'

xgb_accuracy = round(accuracy_score(y_test, preds_xgb), 3)
xgb_accuracy_ci_length = z_value * np.sqrt((xgb_accuracy * (1 - xgb_accuracy)) / y_test.shape[0])
xgb_accuracy_ci_lower = round((xgb_accuracy - xgb_accuracy_ci_length), 3)
xgb_accuracy_ci_upper = round((xgb_accuracy + xgb_accuracy_ci_length), 3)
xgb_accuracy_str = str(xgb_accuracy) + ' (' + str(xgb_accuracy_ci_lower) + ' - ' + str(xgb_accuracy_ci_upper) + ')'

xgb_auroc, xgb_auroc_ci_lower, xgb_auroc_ci_upper = auroc_ci(y_test, probs_xgb)
xgb_auroc = round(xgb_auroc, 3)
xgb_auroc_ci_lower = round(xgb_auroc_ci_lower, 3)
xgb_auroc_ci_upper = round(xgb_auroc_ci_upper, 3)
xgb_auroc_str = str(xgb_auroc) + ' (' + str(xgb_auroc_ci_lower) + ' - ' + str(xgb_auroc_ci_upper) + ')'

xgb_brier = round(brier_score_loss(y_test, probs_xgb), 3)
xgb_brier_ci_length = z_value * np.sqrt((xgb_brier * (1 - xgb_brier)) / y_test.shape[0])
xgb_brier_ci_lower = round((xgb_brier - xgb_brier_ci_length), 3)
xgb_brier_ci_upper = round((xgb_brier + xgb_brier_ci_length), 3)
xgb_brier_str = str(xgb_brier) + ' (' + str(xgb_brier_ci_lower) + ' - ' + str(xgb_brier_ci_upper) + ')'

xgb_results = [xgb_sensitivity_str, xgb_specificity_str, xgb_auprc_str, xgb_accuracy_str, xgb_auroc_str, xgb_brier_str]

print("Sensitivity: ", (xgb_sensitivity_str))
print("Specificity: ", (xgb_specificity_str))
print('AUPRC: ', (xgb_auprc_str))
print('Accuracy: ', (xgb_accuracy_str))
print('AUROC: ', (xgb_auroc_str))
print('Brier Score: ', (xgb_brier_str))

In [ ]:
#Recalculate precision recall curve for plotting purposes.

xgb_precision_curve, xgb_recall_curve, _ = precision_recall_curve(y_test, probs_xgb)

# LightGBM

In [ ]:
#Hyperparameter tuning for LightGBM.

def objective(trial):
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective":  trial.suggest_categorical("objective", ["binary"]),
        "metric": "binary_logloss",
        "verbosity": -1,
        "random_state": 31,
        "boosting_type":  trial.suggest_categorical("boosting_type", ["gbdt"]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)
    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize", sampler = TPESampler(seed=31))
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    lgb_params = {}

    for key, value in trial.params.items():
        lgb_params[key] = value

In [ ]:
#See LightGBM hyperparameters.

lgb_params['metric'] = 'binary_logloss'
lgb_params['verbosity'] = -1
lgb_params['random_state'] = 31

print(lgb_params)

In [ ]:
#Fit LightGBM.

import lightgbm as lgb

lgb = lgb.LGBMClassifier(**lgb_params)

lgb.fit(train_x, train_y)

In [ ]:
#Predict on the validation set, get predicted probabilities for calibration, and fit calibration function.

calib_probs_lgb = lgb.predict_proba(valid_x)

calib_model_lgb = LogisticRegression()
calib_model_lgb.fit(calib_probs_lgb, valid_y)

In [ ]:
#Make predictions on the test set based on the trained and calibrated LightGBM model.

preds_lgb = lgb.predict(x_test)

uncalibrated_probs_lgb = lgb.predict_proba(x_test)
uncalibrated_probs_lgb = uncalibrated_probs_lgb

probs_lgb = calib_model_lgb.predict_proba(uncalibrated_probs_lgb)
probs_lgb = probs_lgb[:, 1]

In [ ]:
#Evaluate LightGBM model.

lgb_tn, lgb_fp, lgb_fn, lgb_tp = confusion_matrix(y_test, preds_lgb).ravel()
lgb_cm = confusion_matrix(y_test, preds_lgb)

lgb_sensitivity = round(lgb_tp/(lgb_tp+lgb_fn), 3)
lgb_sensitivity_ci_length = z_value * np.sqrt((lgb_sensitivity * (1 - lgb_sensitivity)) / y_test.shape[0])
lgb_sensitivity_ci_lower = round((lgb_sensitivity - lgb_sensitivity_ci_length), 3)
lgb_sensitivity_ci_upper = round((lgb_sensitivity + lgb_sensitivity_ci_length), 3)
lgb_sensitivity_str = str(lgb_sensitivity) + ' (' + str(lgb_sensitivity_ci_lower) + ' - ' + str(lgb_sensitivity_ci_upper) + ')'

lgb_specificity = round(lgb_tn/(lgb_tn+lgb_fp), 3)
lgb_specificity_ci_length = z_value * np.sqrt((lgb_specificity * (1 - lgb_specificity)) / y_test.shape[0])
lgb_specificity_ci_lower = round((lgb_specificity - lgb_specificity_ci_length), 3)
lgb_specificity_ci_upper = round((lgb_specificity + lgb_specificity_ci_length), 3)
lgb_specificity_str = str(lgb_specificity) + ' (' + str(lgb_specificity_ci_lower) + ' - ' + str(lgb_specificity_ci_upper) + ')'

lgb_auprc = round(average_precision_score(y_test, probs_lgb), 3)
lgb_auprc_ci_length = z_value * np.sqrt((lgb_auprc * (1 - lgb_auprc)) / y_test.shape[0])
lgb_auprc_ci_lower = round((lgb_auprc - lgb_auprc_ci_length), 3)
lgb_auprc_ci_upper = round((lgb_auprc + lgb_auprc_ci_length), 3)
lgb_auprc_str = str(lgb_auprc) + ' (' + str(lgb_auprc_ci_lower) + ' - ' + str(lgb_auprc_ci_upper) + ')'

lgb_accuracy = round(accuracy_score(y_test, preds_lgb), 3)
lgb_accuracy_ci_length = z_value * np.sqrt((lgb_accuracy * (1 - lgb_accuracy)) / y_test.shape[0])
lgb_accuracy_ci_lower = round((lgb_accuracy - lgb_accuracy_ci_length), 3)
lgb_accuracy_ci_upper = round((lgb_accuracy + lgb_accuracy_ci_length), 3)
lgb_accuracy_str = str(lgb_accuracy) + ' (' + str(lgb_accuracy_ci_lower) + ' - ' + str(lgb_accuracy_ci_upper) + ')'

lgb_auroc, lgb_auroc_ci_lower, lgb_auroc_ci_upper = auroc_ci(y_test, probs_lgb)
lgb_auroc = round(lgb_auroc, 3)
lgb_auroc_ci_lower = round(lgb_auroc_ci_lower, 3)
lgb_auroc_ci_upper = round(lgb_auroc_ci_upper, 3)
lgb_auroc_str = str(lgb_auroc) + ' (' + str(lgb_auroc_ci_lower) + ' - ' + str(lgb_auroc_ci_upper) + ')'

lgb_brier = round(brier_score_loss(y_test, probs_lgb), 3)
lgb_brier_ci_length = z_value * np.sqrt((lgb_brier * (1 - lgb_brier)) / y_test.shape[0])
lgb_brier_ci_lower = round((lgb_brier - lgb_brier_ci_length), 3)
lgb_brier_ci_upper = round((lgb_brier + lgb_brier_ci_length), 3)
lgb_brier_str = str(lgb_brier) + ' (' + str(lgb_brier_ci_lower) + ' - ' + str(lgb_brier_ci_upper) + ')'

lgb_results = [lgb_sensitivity_str, lgb_specificity_str, lgb_auprc_str, lgb_accuracy_str, lgb_auroc_str, lgb_brier_str]

print("Sensitivity: ", (lgb_sensitivity_str))
print("Specificity: ", (lgb_specificity_str))
print('AUPRC: ', (lgb_auprc_str))
print('Accuracy: ', (lgb_accuracy_str))
print('AUROC: ', (lgb_auroc_str))
print('Brier Score: ', (lgb_brier_str))

In [ ]:
#Recalculate precision recall curve for plotting purposes.

lgb_precision_curve, lgb_recall_curve, _ = precision_recall_curve(y_test, probs_lgb)

# Random Forest

In [ ]:
#Hyperparameter tuning for Random Forest.

def objective(trial):

    param = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "random_state": 31,
        "max_features": trial.suggest_categorical("max_features", ["auto", "sqrt","log2", None]),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000, 100),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4, 1),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10, 1),
    }

    rf = RandomForestClassifier(**param)

    rf.fit(
        train_x,
        train_y,
    )

    preds = rf.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=31))
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    rf_params = {}

    for key, value in trial.params.items():
        rf_params[key] = value

In [ ]:
#See Random Forest hyperparameters.

rf_params['random_state'] = 31

print(rf_params)

In [ ]:
#Fit Random Forest.

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(**rf_params)

rf.fit(train_x, train_y)

In [ ]:
#Predict on the validation set, get predicted probabilities for calibration, and fit calibration function.

calib_probs_rf = rf.predict_proba(valid_x)

calib_model_rf = LogisticRegression()
calib_model_rf.fit(calib_probs_rf, valid_y)

In [ ]:
#Make predictions on the test set based on the trained Random Forest model.

preds_rf = rf.predict(x_test)

uncalibrated_probs_rf = rf.predict_proba(x_test)

probs_rf = calib_model_rf.predict_proba(uncalibrated_probs_rf)
probs_rf = probs_rf[:, 1]

In [ ]:
#Evaluate Random Forest model.

rf_tn, rf_fp, rf_fn, rf_tp = confusion_matrix(y_test, preds_rf).ravel()
rf_cm = confusion_matrix(y_test, preds_rf)

rf_sensitivity = round(rf_tp/(rf_tp+rf_fn), 3)
rf_sensitivity_ci_length = z_value * np.sqrt((rf_sensitivity * (1 - rf_sensitivity)) / y_test.shape[0])
rf_sensitivity_ci_lower = round((rf_sensitivity - rf_sensitivity_ci_length), 3)
rf_sensitivity_ci_upper = round((rf_sensitivity + rf_sensitivity_ci_length), 3)
rf_sensitivity_str = str(rf_sensitivity) + ' (' + str(rf_sensitivity_ci_lower) + ' - ' + str(rf_sensitivity_ci_upper) + ')'

rf_specificity = round(rf_tn/(rf_tn+rf_fp), 3)
rf_specificity_ci_length = z_value * np.sqrt((rf_specificity * (1 - rf_specificity)) / y_test.shape[0])
rf_specificity_ci_lower = round((rf_specificity - rf_specificity_ci_length), 3)
rf_specificity_ci_upper = round((rf_specificity + rf_specificity_ci_length), 3)
rf_specificity_str = str(rf_specificity) + ' (' + str(rf_specificity_ci_lower) + ' - ' + str(rf_specificity_ci_upper) + ')'

rf_auprc = round(average_precision_score(y_test, probs_rf), 3)
rf_auprc_ci_length = z_value * np.sqrt((rf_auprc * (1 - rf_auprc)) / y_test.shape[0])
rf_auprc_ci_lower = round((rf_auprc - rf_auprc_ci_length), 3)
rf_auprc_ci_upper = round((rf_auprc + rf_auprc_ci_length), 3)
rf_auprc_str = str(rf_auprc) + ' (' + str(rf_auprc_ci_lower) + ' - ' + str(rf_auprc_ci_upper) + ')'

rf_accuracy = round(accuracy_score(y_test, preds_rf), 3)
rf_accuracy_ci_length = z_value * np.sqrt((rf_accuracy * (1 - rf_accuracy)) / y_test.shape[0])
rf_accuracy_ci_lower = round((rf_accuracy - rf_accuracy_ci_length), 3)
rf_accuracy_ci_upper = round((rf_accuracy + rf_accuracy_ci_length), 3)
rf_accuracy_str = str(rf_accuracy) + ' (' + str(rf_accuracy_ci_lower) + ' - ' + str(rf_accuracy_ci_upper) + ')'

rf_auroc, rf_auroc_ci_lower, rf_auroc_ci_upper = auroc_ci(y_test, probs_rf)
rf_auroc = round(rf_auroc, 3)
rf_auroc_ci_lower = round(rf_auroc_ci_lower, 3)
rf_auroc_ci_upper = round(rf_auroc_ci_upper, 3)
rf_auroc_str = str(rf_auroc) + ' (' + str(rf_auroc_ci_lower) + ' - ' + str(rf_auroc_ci_upper) + ')'

rf_brier = round(brier_score_loss(y_test, probs_rf), 3)
rf_brier_ci_length = z_value * np.sqrt((rf_brier * (1 - rf_brier)) / y_test.shape[0])
rf_brier_ci_lower = round((rf_brier - rf_brier_ci_length), 3)
rf_brier_ci_upper = round((rf_brier + rf_brier_ci_length), 3)
rf_brier_str = str(rf_brier) + ' (' + str(rf_brier_ci_lower) + ' - ' + str(rf_brier_ci_upper) + ')'

rf_results = [rf_sensitivity_str, rf_specificity_str, rf_auprc_str, rf_accuracy_str, rf_auroc_str, rf_brier_str]

print("Sensitivity: ", (rf_sensitivity_str))
print("Specificity: ", (rf_specificity_str))
print('AUPRC: ', (rf_auprc_str))
print('Accuracy: ', (rf_accuracy_str))
print('AUROC: ', (rf_auroc_str))
print('Brier Score: ', (rf_brier_str))

In [ ]:
#Recalculate precision recall curve for plotting purposes.

rf_precision_curve, rf_recall_curve, _ = precision_recall_curve(y_test, probs_rf)

# ROC and PR Curves

In [ ]:
#Plot ROC curve.

f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

tabpfn_fpr, tabpfn_tpr, _ = roc_curve(y_test, probs_tabpfn)
tabpfn_label = 'TabPFN AUROC: ' + tabpfn_auroc_str
pyplot.plot(tabpfn_fpr, tabpfn_tpr, label = tabpfn_label, color = 'b', linewidth = 3.5, alpha = 0.75)

tabnet_fpr, tabnet_tpr, _ = roc_curve(y_test, probs_tabnet)
tabnet_label = 'TabNet AUROC: ' + tabnet_auroc_str
pyplot.plot(tabnet_fpr, tabnet_tpr, label = tabnet_label, color = 'g', linewidth = 3.5, alpha = 0.75)

xgb_fpr, xgb_tpr, _ = roc_curve(y_test, probs_xgb)
xgb_label = 'XGBoost AUROC: ' + xgb_auroc_str
pyplot.plot(xgb_fpr, xgb_tpr, label = xgb_label, color = 'r', linewidth = 3.5, alpha = 0.75)

lgb_fpr, lgb_tpr, _ = roc_curve(y_test, probs_lgb)
lgb_label = 'LightGBM AUROC: ' + lgb_auroc_str
pyplot.plot(lgb_fpr, lgb_tpr, label = lgb_label, color='c', linewidth = 3.5, alpha = 0.75)

rf_fpr, rf_tpr, _ = roc_curve(y_test, probs_rf)
rf_label = 'Random Forest AUROC: ' + rf_auroc_str
pyplot.plot(rf_fpr, rf_tpr, label = rf_label, color = 'm', linewidth = 3.5, alpha = 0.75)

pyplot.plot([0, 1], [0, 1], linestyle = '--', linewidth=2)

pyplot.title('C', x = -0.075, y = 1.005, fontsize = 75, pad = 20)
pyplot.xlabel('False Positive Rate', fontsize = 22, fontweight = 'heavy', labelpad = 16)
pyplot.ylabel('True Positive Rate', fontsize = 22, fontweight = 'heavy', labelpad = 16)
pyplot.tick_params(axis="y",direction="out", labelsize = 16)
pyplot.tick_params(axis="x",direction="out", labelsize = 16)

leg = pyplot.legend(loc = 'lower right', fontsize = 18)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_roc.png', dpi=300)
pyplot.show()

In [ ]:
#Plot PR curve.

f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

tabpfn_label = 'TabPFN AUPRC: ' + tabpfn_auprc_str
tabnet_label = 'TabNet AUPRC: ' + tabnet_auprc_str
xgb_label = 'XGBoost AUPRC: ' + xgb_auprc_str
lgb_label = 'LightGBM AUPRC: ' + lgb_auprc_str
rf_label = 'Random Forest AUPRC: ' + rf_auprc_str

pyplot.plot(tabpfn_recall_curve, tabpfn_precision_curve, label = tabpfn_label, color = 'b', linewidth = 3.5, alpha = 0.75)
pyplot.plot(tabnet_recall_curve, tabnet_precision_curve, label = tabnet_label, color = 'g', linewidth = 3.5, alpha = 0.75)
pyplot.plot(xgb_recall_curve, xgb_precision_curve, label = xgb_label, color = 'r', linewidth = 3.5, alpha = 0.75)
pyplot.plot(lgb_recall_curve, lgb_precision_curve, label = lgb_label, color = 'c', linewidth = 3.5, alpha = 0.75)
pyplot.plot(rf_recall_curve, rf_precision_curve, label = rf_label, color = 'm', linewidth = 3.5, alpha = 0.75)

pyplot.title('C', x = -0.075, y = 1.005, fontsize = 75, pad = 20)
pyplot.xlabel('Recall', fontsize = 18, fontweight = 'heavy', labelpad = 16)
pyplot.ylabel('Precision', fontsize = 22, fontweight = 'heavy', labelpad = 16)
pyplot.tick_params(axis="y",direction="out", labelsize = 16)
pyplot.tick_params(axis="x",direction="out", labelsize = 16)

leg = pyplot.legend(loc = 'upper right', fontsize = 18)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_prc.png', dpi=300)
pyplot.show()

# Results Summary

In [ ]:
results = {'TabPFN':tabpfn_results, 'TabNet':tabnet_results, 'XGBoost':xgb_results, 'LightGBM':lgb_results, 'Random Forest':rf_results}

results = pd.DataFrame(results, columns = ['TabPFN', 'TabNet', 'XGBoost', 'LightGBM', 'Random Forest'])
results = results.T

results.columns = ['Sensitivity (95% CI)', 'Specificity (95% CI)',  'AUPRC (95% CI)', 'Accuracy (95% CI)', 'AUROC (95% CI)', 'Brier Score (95% CI)']

results.to_csv('/content/drive/MyDrive/NCDB-Meningioma/60m_results.csv')

results

# Confusion Matrices

In [ ]:
tabpfn_cm = confusion_matrix(y_test, preds_tabpfn)

f = pyplot.figure(figsize=(8,8))

sns.heatmap(tabpfn_cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Survival', 'Mortality']
pyplot.xticks([0.5,1.5], labels, fontsize=16, fontweight='heavy')
pyplot.yticks([0.5,1.5], labels, fontsize=16, fontweight='heavy', va='center')

pyplot.xlabel('Predicted', fontsize=22, fontweight='heavy', labelpad=16)
pyplot.ylabel('Truth', fontsize=22, fontweight='heavy', labelpad=16)

pyplot.tick_params(axis="y",direction="out", pad=10)
pyplot.tick_params(axis="x",direction="out", pad=10)
pyplot.title('A', x = -0.095, y = 1.005, fontsize = 75, pad = 20)

pyplot.subplots_adjust(left=0.20, right=0.85, bottom=0.20, top=0.80)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_cm_tabpfn.png', dpi=300)
pyplot.show()

In [ ]:
tabnet_cm = confusion_matrix(y_test, preds_tabnet)

f = pyplot.figure(figsize=(8,8))

sns.heatmap(tabnet_cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Survival', 'Mortality']
pyplot.xticks([0.5,1.5], labels, fontsize=16, fontweight='heavy')
pyplot.yticks([0.5,1.5], labels, fontsize=16, fontweight='heavy', va='center')

pyplot.xlabel('Predicted', fontsize=22, fontweight='heavy', labelpad=16)
pyplot.ylabel('Truth', fontsize=22, fontweight='heavy', labelpad=16)

pyplot.tick_params(axis="y",direction="out", pad=10)
pyplot.tick_params(axis="x",direction="out", pad=10)
pyplot.title('B', x = -0.095, y = 1.005, fontsize = 75, pad = 20)

pyplot.subplots_adjust(left=0.20, right=0.85, bottom=0.20, top=0.80)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_cm_tabnet.png', dpi=300)
pyplot.show()

In [ ]:
xgb_cm = confusion_matrix(y_test, preds_xgb)

f = pyplot.figure(figsize=(8,8))

sns.heatmap(xgb_cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Survival', 'Mortality']
pyplot.xticks([0.5,1.5], labels, fontsize=16, fontweight='heavy')
pyplot.yticks([0.5,1.5], labels, fontsize=16, fontweight='heavy', va='center')

pyplot.xlabel('Predicted', fontsize=22, fontweight='heavy', labelpad=16)
pyplot.ylabel('Truth', fontsize=22, fontweight='heavy', labelpad=16)

pyplot.tick_params(axis="y",direction="out", pad=10)
pyplot.tick_params(axis="x",direction="out", pad=10)
pyplot.title('C', x = -0.095, y = 1.005, fontsize = 75, pad = 20)

pyplot.subplots_adjust(left=0.20, right=0.85, bottom=0.20, top=0.80)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_cm_xgb.png', dpi=300)
pyplot.show()

In [ ]:
lgb_cm = confusion_matrix(y_test, preds_lgb)

f = pyplot.figure(figsize=(8,8))

sns.heatmap(lgb_cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Survival', 'Mortality']
pyplot.xticks([0.5,1.5], labels, fontsize=16, fontweight='heavy')
pyplot.yticks([0.5,1.5], labels, fontsize=16, fontweight='heavy', va='center')

pyplot.xlabel('Predicted', fontsize=22, fontweight='heavy', labelpad=16)
pyplot.ylabel('Truth', fontsize=22, fontweight='heavy', labelpad=16)

pyplot.tick_params(axis="y",direction="out", pad=10)
pyplot.tick_params(axis="x",direction="out", pad=10)
pyplot.title('C', x = -0.095, y = 1.005, fontsize = 75, pad = 20)

pyplot.subplots_adjust(left=0.20, right=0.85, bottom=0.20, top=0.80)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_cm_lgb.png', dpi=300)
pyplot.show()

In [ ]:
rf_cm = confusion_matrix(y_test, preds_rf)

f = pyplot.figure(figsize=(8,8))

sns.heatmap(rf_cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 16}, linewidths=1, linecolor='black')

labels = ['Survival', 'Mortality']
pyplot.xticks([0.5,1.5], labels, fontsize=16, fontweight='heavy')
pyplot.yticks([0.5,1.5], labels, fontsize=16, fontweight='heavy', va='center')

pyplot.xlabel('Predicted', fontsize=22, fontweight='heavy', labelpad=16)
pyplot.ylabel('Truth', fontsize=22, fontweight='heavy', labelpad=16)

pyplot.tick_params(axis="y",direction="out", pad=10)
pyplot.tick_params(axis="x",direction="out", pad=10)
pyplot.title('D', x = -0.095, y = 1.005, fontsize = 75, pad = 20)

pyplot.subplots_adjust(left=0.20, right=0.85, bottom=0.20, top=0.80)

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_cm_rf.png', dpi=300)
pyplot.show()

# SHAP Plots

In [ ]:
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_yticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                                    break_long_words=break_long_words))
    ax.set_yticklabels(labels, rotation=0)

In [ ]:
feature_names = x_test.columns

In [ ]:
#Calculate SHAP values for TabPFN.

tabpfn_explainer = shap.Explainer(tabpfn.predict, x_test)
tabpfn_shap_values = tabpfn_explainer(x_test)

In [ ]:
#Plot SHAP bar plot for TabPFN.

shap.plots.bar(tabpfn_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('A', x = -0.5, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_shap_tabpfn.png', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for TabNet.

tabnet_explainer = shap.Explainer(tabnet.predict, x_test.values, feature_names=feature_names)
tabnet_shap_values = tabnet_explainer(x_test.values)

In [ ]:
#Plot SHAP bar plot for TabNet.

shap.plots.bar(tabnet_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('B', x = -0.5, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_shap_tabnet.png', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for XGBoost.

xgb_explainer = shap.Explainer(xgb.predict, x_test)
xgb_shap_values = xgb_explainer(x_test)

In [ ]:
#Plot SHAP bar plot for XGBoost.

shap.plots.bar(xgb_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('C', x = -0.5, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_shap_xgb.png', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for LightGBM.

lgb_explainer = shap.Explainer(lgb.predict, x_test)
lgb_shap_values = lgb_explainer(x_test)

In [ ]:
#Plot SHAP bar plot for LightGBM.

shap.plots.bar(lgb_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('C', x = -0.5, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_shap_lgb.png', dpi=300, bbox_inches='tight')

In [ ]:
#Calculate SHAP values for Random Forest.

rf_explainer = shap.Explainer(rf.predict, x_test.sample(frac=0.10, random_state=42))
rf_shap_values = rf_explainer(x_test.sample(frac=0.10, random_state=42))

In [ ]:
#Plot SHAP bar plot for Random Forest.

shap.plots.bar(rf_shap_values, max_display = 15, show=False)

fig = pyplot.gcf()
ax = pyplot.gca()
fig.set_figheight(12)
fig.set_figwidth(5)

pyplot.title('D', x = -0.5, y = 1, fontsize = 50, pad = 20)
pyplot.xlabel("Mean |SHAP Value|", fontsize =12, fontweight = 'heavy', labelpad = 8)
pyplot.tick_params(axis="y",direction="out", labelsize = 12)
pyplot.tick_params(axis="x",direction="out", labelsize = 12)

wrap_labels(ax, 30)
ax.figure

pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_shap_rf.png', dpi=300, bbox_inches='tight')

#Partial Dependency Plots

In [ ]:
column_filters = {
    'Age': (18, 90),
}

for column, (min_val, max_val) in column_filters.items():
    if column in x_test.columns:
        x_test = x_test[(x_test[column] > min_val) & (x_test[column] < max_val)]

In [ ]:
pyplot.rcParams["figure.figsize"] = (15, 15)
pyplot.rcParams["figure.dpi"] = 300
pyplot.rcParams['axes.labelweight'] = 'bold'
pyplot.rcParams['axes.labelsize'] = 12
pyplot.rcParams['axes.labelpad'] = 6
pyplot.rcParams['font.weight'] = 'normal'
pyplot.rcParams['lines.linewidth'] = 2.5
pyplot.rcParams['xtick.labelsize'] = 8
pyplot.rcParams['ytick.labelsize'] = 8

In [ ]:
categorical_features = ['Grade', 'Histology', 'Facility Type', 'Facility Location', 'Sex', 'Race', 'Hispanic Ethnicity',
                        'Primary Payor', 'Percent No High School Education Quartiles', 'Census Median Income Quartiles',
                        'Rurality', 'Charlson-Deyo Score', 'Laterality', 'Diagnostic Biopsy',
                        'Ki-67 Labeling Index', 'Karnofsky Performance Scale', 'Focality', 'Tumor Size',
                        'Extent of Resection', 'Resective Surgery', 'Radiation Treatment']

In [ ]:
#Calculate the mean absolute SHAP values for each feature.
tabpfn_mean_abs_shap_values = np.mean(np.abs(tabpfn_shap_values.values), axis=0)

#Create a DataFrame to map feature names to their mean absolute SHAP values.
tabpfn_shap_summary = pd.DataFrame(list(zip(feature_names, tabpfn_mean_abs_shap_values)), columns=['Feature', 'Mean SHAP'])

#Sort the DataFrame by 'Mean SHAP' in descending order.
tabpfn_shap_summary_sorted = tabpfn_shap_summary.sort_values('Mean SHAP', ascending=False)

#Get the names of the features.
tabpfn_features = tabpfn_shap_summary_sorted['Feature'].tolist()
tabpfn_features = tabpfn_features[:9]

tabpfn_categorical_features = [item for item in tabpfn_features if item in categorical_features]

PartialDependenceDisplay.from_estimator(tabpfn, x_test.sample(frac=0.05, random_state=42), tabpfn_features, categorical_features = tabpfn_categorical_features)
pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_pdp_tabpfn.png', dpi=300, bbox_inches='tight')
pyplot.show()

In [ ]:
#Calculate the mean absolute SHAP values for each feature.
tabnet_mean_abs_shap_values = np.mean(np.abs(tabnet_shap_values.values), axis=0)

#Create a DataFrame to map feature names to their mean absolute SHAP values.
tabnet_shap_summary = pd.DataFrame(list(zip(feature_names, tabnet_mean_abs_shap_values)), columns=['Feature', 'Mean SHAP'])

#Sort the DataFrame by 'Mean SHAP' in descending order.
tabnet_shap_summary_sorted = tabnet_shap_summary.sort_values('Mean SHAP', ascending=False)

#Get the names of the features.
tabnet_features = tabnet_shap_summary_sorted['Feature'].tolist()
tabnet_features = tabnet_features[:9]

tabnet_categorical_features = [item for item in tabnet_features if item in categorical_features]

PartialDependenceDisplay.from_estimator(xgb, x_test, tabnet_features, categorical_features = tabnet_categorical_features)
pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_pdp_tabnet.png', dpi=300, bbox_inches='tight')
pyplot.show()

In [ ]:
#Calculate the mean absolute SHAP values for each feature.
xgb_mean_abs_shap_values = np.mean(np.abs(xgb_shap_values.values), axis=0)

#Create a DataFrame to map feature names to their mean absolute SHAP values.
xgb_shap_summary = pd.DataFrame(list(zip(feature_names, xgb_mean_abs_shap_values)), columns=['Feature', 'Mean SHAP'])

#Sort the DataFrame by 'Mean SHAP' in descending order.
xgb_shap_summary_sorted = xgb_shap_summary.sort_values('Mean SHAP', ascending=False)

#Get the names of the features.
xgb_features = xgb_shap_summary_sorted['Feature'].tolist()
xgb_features = xgb_features[:9]

xgb_categorical_features = [item for item in xgb_features if item in categorical_features]

PartialDependenceDisplay.from_estimator(xgb, x_test, xgb_features, categorical_features = xgb_categorical_features)
pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_pdp_xgb.png', dpi=300, bbox_inches='tight')
pyplot.show()

In [ ]:
#Calculate the mean absolute SHAP values for each feature.
lgb_mean_abs_shap_values = np.mean(np.abs(lgb_shap_values.values), axis=0)

#Create a DataFrame to map feature names to their mean absolute SHAP values.
lgb_shap_summary = pd.DataFrame(list(zip(feature_names, lgb_mean_abs_shap_values)), columns=['Feature', 'Mean SHAP'])

#Sort the DataFrame by 'Mean SHAP' in descending order.
lgb_shap_summary_sorted = lgb_shap_summary.sort_values('Mean SHAP', ascending=False)

#Get the names of the features.
lgb_features = lgb_shap_summary_sorted['Feature'].tolist()
lgb_features = lgb_features[:9]

lgb_categorical_features = [item for item in lgb_features if item in categorical_features]

PartialDependenceDisplay.from_estimator(lgb, x_test, lgb_features, categorical_features = lgb_categorical_features)
pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_pdp_lgb.png', dpi=300, bbox_inches='tight')
pyplot.show()

In [ ]:
#Calculate the mean absolute SHAP values for each feature.
rf_mean_abs_shap_values = np.mean(np.abs(rf_shap_values.values), axis=0)

#Create a DataFrame to map feature names to their mean absolute SHAP values.
rf_shap_summary = pd.DataFrame(list(zip(feature_names, rf_mean_abs_shap_values)), columns=['Feature', 'Mean SHAP'])

#Sort the DataFrame by 'Mean SHAP' in descending order.
rf_shap_summary_sorted = rf_shap_summary.sort_values('Mean SHAP', ascending=False)

#Get the names of the features.
rf_features = rf_shap_summary_sorted['Feature'].tolist()
rf_features = rf_features[:9]

rf_categorical_features = [item for item in rf_features if item in categorical_features]

PartialDependenceDisplay.from_estimator(rf, x_test, rf_features, categorical_features = rf_categorical_features)
pyplot.savefig('/content/drive/MyDrive/NCDB-Meningioma/60m_pdp_rf.png', dpi=300, bbox_inches='tight')
pyplot.show()